In [ ]:
import pandas as pd
import os
import numpy as np
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [ ]:
os.getcwd()

In [ ]:
# Raw data from Compustat
fund_data = pd.read_csv("../data/unprocessed/global_fundamentals_all_annual.zip", engine="pyarrow")

In [ ]:
fund_data["datadate"] = pd.to_datetime(fund_data["datadate"])
fund_data.sort_values(["datadate", "gvkey"], inplace=True)

In [ ]:
fund_data = fund_data[fund_data["curcd"].notna()]

In [ ]:
fund_data[["gvkey", "datadate"]].value_counts().head()

In [ ]:
fundamentals_feature_dict = {
    'gvkey': 'gvkey',
    'conm' : 'company_name',
    'datadate': 'date',
    'fyr': 'fiscal_year_end_month',
    'acctstd': 'accounting_standard_code',
    'bspr': 'best_practice_code',
    'compst': 'company_status_code',
    'curcd': 'currency_code',
    'fyear': 'fiscal_year',
    'scf': 'source_code',
    'src': 'source_code_change',
    'stalt': 'statement_type_code',
    'upd': 'update_code',
    'datadate': 'date',
    'fdate': 'fiscal_date',
    "cshpria" : "shares_eps",
    'pdate': 'period_date'}


global_fundamentals_data = fund_data.copy()

global_fundamentals_features = pd.read_csv('../scripts/data/global_fundamental_features_annual.csv', delimiter=';')
global_fundamentals_features = global_fundamentals_features[global_fundamentals_features['wrds_feature'].notnull()]

all_global_fundamental_features = list(fundamentals_feature_dict.keys()) + (global_fundamentals_features['wrds_name'].str.lower().tolist())
all_global_fundamental_feature_new_names = list(fundamentals_feature_dict.values()) + (global_fundamentals_features['wrds_feature'].tolist())
global_fundamentals_data = global_fundamentals_data[all_global_fundamental_features]
global_fundamentals_data.columns = all_global_fundamental_feature_new_names


In [ ]:
global_fundamentals_data.columns

In [ ]:
global_fundamentals_data['net_income'] = global_fundamentals_data['income_bex'] + global_fundamentals_data['extra']
global_fundamentals_data['net_income'] = global_fundamentals_data['net_income'].fillna(global_fundamentals_data['income_bex'])

In [ ]:
global_fundamentals_data = global_fundamentals_data.sort_values("date")

In [ ]:
forex_data_12m = pd.read_parquet("../data/forex/forex_data_12m.parquet")
forex_data_12m["date"] = pd.to_datetime(forex_data_12m["date"])
forex_data_12m["currency"] = forex_data_12m["currency"].astype("category")
forex_data_12m = forex_data_12m.sort_values("date")


In [ ]:
forex_12m_dict = forex_data_12m.set_index(['date', 'currency'])['to_usd'].to_dict()

In [ ]:
forex_data = pd.read_parquet("../data/forex/forex_data_daily.parquet")
forex_data["date"] = pd.to_datetime(forex_data["date"])
forex_data["currency"] = forex_data["currency"].astype("category")
forex_data = forex_data.sort_values("date")


In [ ]:
forex_dict = forex_data.set_index(['date', 'currency'])['to_usd'].to_dict()

In [ ]:
def convert_to_usd(group):
    date, currency = group.name
    if (date, currency) not in forex_dict:
        print("No currency rate on this date")
        print(date, currency)
        tries = 0
        while (date, currency) not in forex_dict:
            date = date - pd.Timedelta(1, "d")
            tries += 1
            if tries > 10:
                print(group.shape, "affected")
                group.loc[:] = np.nan
                return group
        print(date, "worked")
    to_usd_rate = forex_dict[(date, currency)]
    return group * to_usd_rate

In [ ]:
def convert_to_usd_12m(group):
    date, currency = group.name
    if (date, currency) not in forex_12m_dict:
        print("No currency rate on this date")
        print(date, currency)
        tries = 0
        while (date, currency) not in forex_12m_dict:
            date = date - pd.Timedelta(1, "d")
            tries += 1
            if tries > 10:
                print(group.shape, "affected")
                group.loc[:] = np.nan
                return group
        print(date, "worked")
    to_usd_rate = forex_12m_dict[(date, currency)]
    return group * to_usd_rate

In [ ]:
global_fundamentals_data.sort_values(["date", "gvkey"], inplace=True)
global_fundamentals_data.reset_index(inplace=True)

In [ ]:
#CURRENCY. TO USD
for feature in global_fundamentals_features["wrds_feature"]:
    print(feature)
    print(global_fundamentals_features[global_fundamentals_features["wrds_feature"] == feature]["TTM"].item())
    if global_fundamentals_features[global_fundamentals_features["wrds_feature"] == feature]["TTM"].item() == "sum":
        result = global_fundamentals_data.groupby(["date", "currency_code"])[feature].transform(convert_to_usd_12m)
        global_fundamentals_data[feature] = global_fundamentals_data.groupby(["date", "currency_code"])[feature].transform(convert_to_usd_12m)
    elif global_fundamentals_features[global_fundamentals_features["wrds_feature"] == feature]["TTM"].item() == "no":
        global_fundamentals_data[feature] = global_fundamentals_data.groupby(["date", "currency_code"])[feature].transform(convert_to_usd)

In [ ]:
fundamentals_data = global_fundamentals_data.copy()

fundamentals_data['net_debt'] = fundamentals_data['short_debt'] + fundamentals_data['long_debt'] - fundamentals_data['cash_and_eq']
fundamentals_data['current_ratio'] = fundamentals_data['current_assets'] / fundamentals_data['current_liabilites']
fundamentals_data['quick_ratio'] = (fundamentals_data['current_assets'] - fundamentals_data['inventories']) / fundamentals_data['current_liabilites']
fundamentals_data['cash_ratio'] = fundamentals_data['cash_and_eq'] / fundamentals_data['current_liabilites']
fundamentals_data['total_assets_to_liabilites'] = fundamentals_data['total_assets'] / fundamentals_data['total_liabilites']
fundamentals_data['equity_to_debt_ratio'] = fundamentals_data['stockholders_equity'] / fundamentals_data['total_liabilites']
fundamentals_data['interest_coverage_ratio'] = fundamentals_data['op_income_ad'] / fundamentals_data['interest_expense']
fundamentals_data['debt_service_coverage_ratio'] = fundamentals_data['op_income_ad'] / fundamentals_data['short_debt']
fundamentals_data['asset_turnover_ratio'] = fundamentals_data['net_sales'] / fundamentals_data['total_assets']
fundamentals_data['inventory_turnover_ratio'] = fundamentals_data['cost_goods_sold'] / fundamentals_data['inventories']
fundamentals_data['operating_margin_ratio'] = fundamentals_data['op_income_ad'] / fundamentals_data['net_sales']
fundamentals_data['return_on_assets'] = fundamentals_data['net_income'] / fundamentals_data['total_assets']
fundamentals_data['return_on_equity'] = fundamentals_data['net_income'] / fundamentals_data['stockholders_equity']
fundamentals_data['EBITDA'] = fundamentals_data['ebitda']
fundamentals_data['EBITDA_to_net_debt'] = fundamentals_data['EBITDA'] / fundamentals_data['net_debt']
fundamentals_data['EBITDA_to_interest_expense'] = fundamentals_data['EBITDA'] / fundamentals_data['interest_expense']
fundamentals_data['total_assets_to_debt'] = fundamentals_data['total_assets'] / (fundamentals_data['short_debt'] + fundamentals_data['long_debt'])
fundamentals_data['gross_margin'] = (fundamentals_data['net_sales'] - fundamentals_data['cost_goods_sold'])

global_fundamentals_data = fundamentals_data

In [ ]:
global_fundamentals_data.drop(columns=["index"], inplace=True)

In [ ]:
global_fundamentals_data.to_parquet("../data/processed/global_fundamentals_annual_processed.parquet", index=False)

# CSRP (US)

#### No currency change needed

In [ ]:
fund_data = pd.read_csv("../data/unprocessed/csrp/csrp_fundamentals_annual.zip", engine="pyarrow")
fund_data["datadate"] = pd.to_datetime(fund_data["datadate"])
fund_data.columns = [x.lower() for x in fund_data.columns]

In [ ]:
fund_data = fund_data[fund_data["linkprim"].isin(["P", "C"])]

In [ ]:
fundamentals_feature_dict = {
    'gvkey': 'gvkey',
    'lpermco' : 'permco',
    'lpermno' : 'permno',
    'liid' : 'liid',
    'linkdt' : 'linkdt',
    'linkenddt' : 'linkenddt',
    'linkprim' : 'primary_link',
    'linktype' : 'link_type',
    'conm' : 'company_name',
    'datadate': 'date',
    'fyr': 'fiscal_year_end_month',
    'acctstd': 'accounting_standard_code',
    'bspr': 'best_practice_code',
    'compst': 'company_status_code',
    'curcd': 'currency_code',
    'fyear': 'fiscal_year',
    'scf': 'source_code',
    'src': 'source_code_change',
    'stalt': 'statement_type_code',
    'upd': 'update_code',
    'datadate': 'date',
    'fdate': 'fiscal_date',
    'pdate': 'period_date'}


us_fundamentals_features = pd.read_csv('../scripts/data/us_fundamental_features_csrp_annual.csv', delimiter=';')
us_fundamentals_features = us_fundamentals_features[us_fundamentals_features['wrds_feature'].notnull()]

all_us_fundamental_features = list(fundamentals_feature_dict.keys()) + (us_fundamentals_features['wrds_name'].str.lower().tolist())
all_us_fundamental_feature_new_names = list(fundamentals_feature_dict.values()) + (us_fundamentals_features['wrds_feature'].tolist())
fund_data = fund_data[all_us_fundamental_features]
fund_data.columns = all_us_fundamental_feature_new_names

In [ ]:
fund_data.sort_values(["date", "gvkey"], inplace=True)

In [ ]:
fund_data['net_income'] = fund_data['net_income'].fillna(fund_data['income_bex'])

In [ ]:
fundamentals_data = fund_data.copy()

fundamentals_data['net_debt'] = fundamentals_data['short_debt'] + fundamentals_data['long_debt'] - fundamentals_data['cash_and_eq']
fundamentals_data['current_ratio'] = fundamentals_data['current_assets'] / fundamentals_data['current_liabilites']
fundamentals_data['quick_ratio'] = (fundamentals_data['current_assets'] - fundamentals_data['inventories']) / fundamentals_data['current_liabilites']
fundamentals_data['cash_ratio'] = fundamentals_data['cash_and_eq'] / fundamentals_data['current_liabilites']
fundamentals_data['total_assets_to_liabilites'] = fundamentals_data['total_assets'] / fundamentals_data['total_liabilites']
fundamentals_data['equity_to_debt_ratio'] = fundamentals_data['stockholders_equity'] / fundamentals_data['total_liabilites']
fundamentals_data['interest_coverage_ratio'] = fundamentals_data['op_income_ad'] / fundamentals_data['interest_expense']
fundamentals_data['debt_service_coverage_ratio'] = fundamentals_data['op_income_ad'] / fundamentals_data['short_debt']
fundamentals_data['asset_turnover_ratio'] = fundamentals_data['net_sales'] / fundamentals_data['total_assets']
fundamentals_data['inventory_turnover_ratio'] = fundamentals_data['cost_goods_sold'] / fundamentals_data['inventories']
fundamentals_data['operating_margin_ratio'] = fundamentals_data['op_income_ad'] / fundamentals_data['net_sales']
fundamentals_data['return_on_assets'] = fundamentals_data['net_income'] / fundamentals_data['total_assets']
fundamentals_data['return_on_equity'] = fundamentals_data['net_income'] / fundamentals_data['stockholders_equity']
fundamentals_data['EBITDA'] = fundamentals_data['ebitda']
fundamentals_data['EBITDA_to_net_debt'] = fundamentals_data['EBITDA'] / fundamentals_data['net_debt']
fundamentals_data['EBITDA_to_interest_expense'] = fundamentals_data['EBITDA'] / fundamentals_data['interest_expense']
fundamentals_data['total_assets_to_debt'] = fundamentals_data['total_assets'] / (fundamentals_data['short_debt'] + fundamentals_data['long_debt'])
fundamentals_data['gross_margin'] = (fundamentals_data['net_sales'] - fundamentals_data['cost_goods_sold'])

fund_data = fundamentals_data

In [ ]:
fund_data.to_parquet('../data/processed/csrp/csrp_compustat_fundamentals_processed_annual.parquet', index=False)